## Part II Project Notebook

### NMA Exploration

In [1]:
%pip install prody
%pip install matplotlib.pylab


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement matplotlib.pylab (from versions: none)
ERROR: No matching distribution found for matplotlib.pylab


### Parse Structure

In [2]:
from prody import *

from matplotlib.pylab import *

ion()

C:\Users\Mihailo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\Bio\pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [3]:
ubi = parsePDB('1aar')

ubi

@> Connecting wwPDB FTP server RCSB PDB (USA).
@> 1aar downloaded (c:\Users\Mihailo\...\1aar.pdb.gz)
@> PDB download via FTP completed (1 downloaded, 0 failed).
@> 1218 atoms and 1 coordinate set(s) were parsed in 0.01s.


<AtomGroup: 1aar (1218 atoms)>

In [4]:
calphas = ubi.select('calpha and chain A and resnum < 71')

calphas

<Selection: 'calpha and chai...and resnum < 71' from 1aar (70 atoms)>

In [5]:
gnm = GNM(name = 'Ubiquintin')

### Build Kirchhoff matrix

In [6]:
gnm.buildKirchhoff(coords=calphas, cutoff=10.0, gamma=1.0)

@> Kirchhoff was built in 0.00s.


In [7]:
gnm.getKirchhoff()

array([[11., -1., -1., ...,  0.,  0.,  0.],
       [-1., 15., -1., ...,  0.,  0.,  0.],
       [-1., -1., 20., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ..., 20., -1., -1.],
       [ 0.,  0.,  0., ..., -1., 21., -1.],
       [ 0.,  0.,  0., ..., -1., -1., 12.]])

### Parameters

In [8]:
gnm.getCutoff()

10.0

In [9]:
gnm.getGamma()

1.0

### Calculate Normal Modes

In [11]:
gnm.calcModes(20, zeros=False)

@> 20 modes were calculated in 0.01s.


### Normal Mode Data

In [12]:
gnm.getEigvals().round(3)

array([ 2.502,  2.812,  4.366,  5.05 ,  7.184,  7.65 ,  7.877,  9.08 ,
        9.713, 10.132, 10.502, 10.644, 10.888, 11.157, 11.285, 11.632,
       11.78 , 11.936, 12.006, 12.218])

In [13]:
gnm.getEigvecs().round(3)

array([[-0.064, -0.131, -0.245, ..., -0.256,  0.538, -0.   ],
       [-0.073, -0.085, -0.19 , ...,  0.006, -0.069,  0.032],
       [-0.076, -0.043, -0.135, ...,  0.017, -0.047,  0.018],
       ...,
       [-0.092,  0.064,  0.105, ...,  0.032, -0.042,  0.006],
       [-0.07 ,  0.099,  0.054, ...,  0.031,  0.024, -0.014],
       [-0.081,  0.135,  0.124, ...,  0.013, -0.04 , -0.018]])

### NMA on CPU Function

In [14]:
def calculate_modes_cpu(protein_name, cutoff = 7.0, num_modes=20, gamma = 1):
    """
    Function to calculate normal modes on CPU

    Arguments:  
        protein_name : string - PDB id of a protein
        cutoff : double - cutoff distance for GNM algorithm
        num_modes : integer - number of lowest non-trivial modes calculated
        gamma : double - constant for potential energy calculation
    Outputs:
        (eigenvals, eigenvecs) : (array(double), array(array(double))) - calculated modes and eigenvalues 
    """
    protein = parsePDB(protein_name)
    calphas = protein.select('calpha')

    gnm = GNM()
    gnm.buildKirchhoff(coords=calphas, cutoff=cutoff, gamma=gamma)

    gnm.calcModes(num_modes, zeros=False)
    return (gnm.getEigvals(), gnm.getEigvecs())

### Downloading PDB Data

In [15]:
%%bash
mkdir pdb
cd pdb
mkdir zipped unzipped
cd zipped
wget ftp://ftp.wwpdb.org/pub/pdb/data/structures/all/pdb/*

--2022-11-21 09:37:23--  ftp://ftp.wwpdb.org/pub/pdb/data/structures/all/pdb/*
           => ‘.listing’
Resolving ftp.wwpdb.org (ftp.wwpdb.org)... 128.6.158.66
Connecting to ftp.wwpdb.org (ftp.wwpdb.org)|128.6.158.66|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/pdb/data/structures/all/pdb ... done.
==> PASV ... done.    ==> LIST ... done.

     0K .......... .......... .......... .......... ..........  607 
    50K .......... .......... .......... .......... ..........  588K
   100K .......... .......... .......... .......... ..........  754K
   150K .......... .......... .......... .......... .......... 1.06M
   200K .......... .......... .......... .......... ..........  626K
   250K .......... .......... .......... .......... ..........  758K
   300K .......... .......... .......... .......... .......... 1.29M
   350K .......... .......... .......... .......... .......... 1.02M
   400K .......